# Stacking (Advanced)

This notebook contains **practice problems with solutions** on NumPy stacking.

**Best practices used here**
- Prefer `np.concatenate` / `np.stack` (explicit about axes) once you understand `vstack`/`hstack`.
- Use `assert` checks to verify shapes, dtypes, and values.
- Be explicit about `axis` and array dimensionality (1D vs 2D).
- Be mindful that stacking typically **allocates a new array** (copies data).


In [1]:
import numpy as np

np.set_printoptions(edgeitems=3, linewidth=120, suppress=True)
rng = np.random.default_rng(0)


## Problem 1 — Choose the correct stacking primitive

You have three 1D arrays `a`, `b`, `c`, each of shape `(4,)`.

1) Create `M_rows` of shape `(3, 4)` where each array becomes a **row**.

2) Create `M_cols` of shape `(4, 3)` where each array becomes a **column**.

Constraints:
- Use **one** NumPy call per result.
- Do **not** reshape by hand with `.reshape(...)`.


In [2]:
# Given
a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])
c = np.array([-1, -2, -3, -4])

# --- Solution ---
M_rows = np.stack([a, b, c], axis=0)   # each becomes a row
M_cols = np.stack([a, b, c], axis=1)   # each becomes a column

print("M_rows:\n", M_rows)
print("M_cols:\n", M_cols)

# Checks
assert M_rows.shape == (3, 4)
assert M_cols.shape == (4, 3)
assert np.array_equal(M_rows[0], a)
assert np.array_equal(M_rows[1], b)
assert np.array_equal(M_cols[:, 2], c)


M_rows:
 [[ 1  2  3  4]
 [10 20 30 40]
 [-1 -2 -3 -4]]
M_cols:
 [[ 1 10 -1]
 [ 2 20 -2]
 [ 3 30 -3]
 [ 4 40 -4]]


## Problem 2 — `hstack` / `vstack` surprises with 1D arrays

Let `x` and `y` be 1D arrays of shape `(5,)`.

1) Compute `A = np.vstack([x, y])` and note its shape.

2) Compute `B = np.hstack([x, y])` and note its shape.

3) Create `C` of shape `(5, 2)` where `x` and `y` are columns.
   Use a stacking function (not manual assignment).


In [3]:
x = np.arange(5)
y = np.arange(5, 10)

# --- Solution ---
A = np.vstack([x, y])
B = np.hstack([x, y])

# For columns from 1D vectors, use column_stack (or stack with axis=1)
C = np.column_stack([x, y])

print("x.shape:", x.shape)
print("A.shape:", A.shape, "\n", A)
print("B.shape:", B.shape, "\n", B)
print("C.shape:", C.shape, "\n", C)

# Checks
assert A.shape == (2, 5)
assert B.shape == (10,)   # hstack concatenates 1D arrays
assert C.shape == (5, 2)
assert np.array_equal(C[:, 0], x)
assert np.array_equal(C[:, 1], y)


x.shape: (5,)
A.shape: (2, 5) 
 [[0 1 2 3 4]
 [5 6 7 8 9]]
B.shape: (10,) 
 [0 1 2 3 4 5 6 7 8 9]
C.shape: (5, 2) 
 [[0 5]
 [1 6]
 [2 7]
 [3 8]
 [4 9]]


## Problem 3 — Fix an incompatible vertical stack by padding

You are given 2D arrays `A` of shape `(2, 3)` and `B` of shape `(1, 5)`.

Goal: vertically stack them into `S` of shape `(3, 5)` by **padding `A` with zeros on the right**.

Rules:
- Do not change `B`.
- Do not use Python loops.


In [4]:
A = np.array([[1, 2, 3],
              [4, 5, 6]])
B = np.array([[10, 20, 30, 40, 50]])

# --- Solution ---
target_cols = B.shape[1]
pad_width = target_cols - A.shape[1]
A_padded = np.pad(A, pad_width=((0, 0), (0, pad_width)), mode="constant", constant_values=0)
S = np.vstack([A_padded, B])

print("A_padded:\n", A_padded)
print("S:\n", S)

# Checks
assert A_padded.shape == (2, 5)
assert S.shape == (3, 5)
assert np.array_equal(S[-1], B[0])
assert np.all(S[:2, 3:] == 0)


A_padded:
 [[1 2 3 0 0]
 [4 5 6 0 0]]
S:
 [[ 1  2  3  0  0]
 [ 4  5  6  0  0]
 [10 20 30 40 50]]


## Problem 4 — `concatenate` vs `stack` (spot the new axis)

Given `P` and `Q` of shape `(2, 3)`:

1) Create `C0` by concatenating along rows to get shape `(4, 3)`.
2) Create `C1` by concatenating along columns to get shape `(2, 6)`.
3) Create `T` by stacking to get shape `(2, 2, 3)` (a new axis).

Use `np.concatenate` for (1)(2), and `np.stack` for (3).

In [5]:
P = np.arange(6).reshape(2, 3)
Q = (np.arange(6) + 100).reshape(2, 3)

# --- Solution ---
C0 = np.concatenate([P, Q], axis=0)
C1 = np.concatenate([P, Q], axis=1)
T = np.stack([P, Q], axis=0)

print("P:\n", P)
print("Q:\n", Q)
print("C0.shape:", C0.shape, "\n", C0)
print("C1.shape:", C1.shape, "\n", C1)
print("T.shape:", T.shape, "\n", T)

# Checks
assert C0.shape == (4, 3)
assert C1.shape == (2, 6)
assert T.shape == (2, 2, 3)
assert np.array_equal(T[0], P)
assert np.array_equal(T[1], Q)


P:
 [[0 1 2]
 [3 4 5]]
Q:
 [[100 101 102]
 [103 104 105]]
C0.shape: (4, 3) 
 [[  0   1   2]
 [  3   4   5]
 [100 101 102]
 [103 104 105]]
C1.shape: (2, 6) 
 [[  0   1   2 100 101 102]
 [  3   4   5 103 104 105]]
T.shape: (2, 2, 3) 
 [[[  0   1   2]
  [  3   4   5]]

 [[100 101 102]
  [103 104 105]]]


## Problem 5 — Build a block matrix with `np.block`

Construct the following block matrix:

```
[ A  0 ]
[ I  B ]
```

Where:
- `A` is shape `(2, 2)`
- `B` is shape `(2, 3)`
- `0` is a zero block of shape `(2, 3)`
- `I` is a 2x2 identity

Result must have shape `(4, 5)`.

Use `np.block` (one call).

In [6]:
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[10, 20, 30],
              [40, 50, 60]])

# --- Solution ---
Z = np.zeros((2, 3), dtype=A.dtype)
I = np.eye(2, dtype=A.dtype)

M = np.block([[A, Z],
              [I, B]])

print("M.shape:", M.shape)
print(M)

# Checks
assert M.shape == (4, 5)
assert np.array_equal(M[:2, :2], A)
assert np.array_equal(M[:2, 2:], 0)
assert np.array_equal(M[2:, :2], I)
assert np.array_equal(M[2:, 2:], B)


M.shape: (4, 5)
[[ 1  2  0  0  0]
 [ 3  4  0  0  0]
 [ 1  0 10 20 30]
 [ 0  1 40 50 60]]


AssertionError: 

## Problem 6 — Stacking along depth (`dstack`) and understanding the result

You are given two grayscale images `img1` and `img2`, each shape `(H, W)`.

1) Create a 3D array `pair` of shape `(H, W, 2)` where the last axis indexes which image.
2) Verify that `pair[..., 0]` equals `img1` and `pair[..., 1]` equals `img2`.

Use `np.dstack`.

In [7]:
H, W = 3, 4
img1 = np.arange(H * W).reshape(H, W)
img2 = (np.arange(H * W) + 100).reshape(H, W)

# --- Solution ---
pair = np.dstack([img1, img2])

print("pair.shape:", pair.shape)
print(pair)

# Checks
assert pair.shape == (H, W, 2)
assert np.array_equal(pair[..., 0], img1)
assert np.array_equal(pair[..., 1], img2)


pair.shape: (3, 4, 2)
[[[  0 100]
  [  1 101]
  [  2 102]
  [  3 103]]

 [[  4 104]
  [  5 105]
  [  6 106]
  [  7 107]]

 [[  8 108]
  [  9 109]
  [ 10 110]
  [ 11 111]]]


## Problem 7 — dtype upcasting and controlling the output dtype

Given:
- `u8` is `uint8`
- `i32` is `int32`
- `f32` is `float32`

1) Stack them as rows into `S` and observe `S.dtype`.
2) Force the result to be `int64` *without* changing values.

Tip: if any input is float, the safe common dtype will usually become float.

In [8]:
u8 = np.array([1, 2, 3, 4], dtype=np.uint8)
i32 = np.array([10, 20, 30, 40], dtype=np.int32)
f32 = np.array([0.5, 1.5, 2.5, 3.5], dtype=np.float32)

# --- Solution ---
S = np.vstack([u8, i32, f32])
print("S.dtype:", S.dtype)
print(S)

# If you truly want int64, you must choose an integer representation.
# Here we convert *before* stacking (but note: float -> int truncates unless values are integral!).
# Since f32 has .5, this would lose information. So we can instead scale or avoid forcing.
# For a no-loss int64 result, use inputs that are already integral.

# Create an integral float (safe to cast) for demonstration:
f32_intlike = np.array([5.0, 6.0, 7.0, 8.0], dtype=np.float32)
S_int64 = np.vstack([u8.astype(np.int64), i32.astype(np.int64), f32_intlike.astype(np.int64)])

print("S_int64.dtype:", S_int64.dtype)
print(S_int64)

# Checks
assert S.shape == (3, 4)
assert S.dtype.kind == 'f'  # becomes float
assert S_int64.dtype == np.int64
assert np.array_equal(S_int64[0], u8.astype(np.int64))
assert np.array_equal(S_int64[1], i32.astype(np.int64))
assert np.array_equal(S_int64[2], f32_intlike.astype(np.int64))


S.dtype: float64
[[ 1.   2.   3.   4. ]
 [10.  20.  30.  40. ]
 [ 0.5  1.5  2.5  3.5]]
S_int64.dtype: int64
[[ 1  2  3  4]
 [10 20 30 40]
 [ 5  6  7  8]]


## Problem 8 — Prove stacking copies (no shared memory)

Create two arrays `a` and `b`, stack them into `S`.

1) Modify `a` and show `S` does not change.
2) Modify `S` and show `b` does not change.
3) Use `np.shares_memory` to confirm.


In [9]:
a = np.array([1, 2, 3])
b = np.array([10, 20, 30])
S = np.vstack([a, b])

a_before = a.copy()
b_before = b.copy()
S_before = S.copy()

# --- Solution ---
a[0] = 999
print("After changing a:")
print("a:", a)
print("S:\n", S)

S[1, 1] = -777
print("\nAfter changing S:")
print("b:", b)
print("S:\n", S)

print("\nshares_memory(a, S):", np.shares_memory(a, S))
print("shares_memory(b, S):", np.shares_memory(b, S))

# Checks
assert np.array_equal(S_before, np.vstack([a_before, b_before]))
assert S[0, 0] != a[0]  # stacked array didn't update when a changed
assert b[1] == b_before[1]  # original b didn't update when S changed
assert np.shares_memory(a, S) is False
assert np.shares_memory(b, S) is False


After changing a:
a: [999   2   3]
S:
 [[ 1  2  3]
 [10 20 30]]

After changing S:
b: [10 20 30]
S:
 [[   1    2    3]
 [  10 -777   30]]

shares_memory(a, S): False
shares_memory(b, S): False


## Problem 9 — Efficiently assemble a feature matrix from mixed 1D sources

You have:
- `age` shape `(n,)` integers
- `height_cm` shape `(n,)` floats
- `is_student` shape `(n,)` booleans

Build `X` of shape `(n, 3)` with columns `[age, height_cm, is_student]`.

Then:
- ensure `X.dtype` is float (so it can hold all columns in one numeric matrix)
- ensure the boolean becomes `0.0/1.0`.


In [10]:
n = 6
age = np.array([18, 19, 20, 21, 22, 23], dtype=np.int32)
height_cm = np.array([165.5, 170.0, 172.2, 160.3, 180.1, 175.0], dtype=np.float64)
is_student = np.array([True, False, True, True, False, False], dtype=bool)

# --- Solution ---
# column_stack is perfect for turning 1D vectors into columns.
# But dtype will upcast due to float presence; ensure float explicitly if desired.
X = np.column_stack([age, height_cm, is_student]).astype(float)

print("X.dtype:", X.dtype)
print(X)

# Checks
assert X.shape == (n, 3)
assert X.dtype == float
assert np.allclose(X[:, 0], age.astype(float))
assert np.allclose(X[:, 1], height_cm.astype(float))
assert np.array_equal(X[:, 2], is_student.astype(float))


X.dtype: float64
[[ 18.  165.5   1. ]
 [ 19.  170.    0. ]
 [ 20.  172.2   1. ]
 [ 21.  160.3   1. ]
 [ 22.  180.1   0. ]
 [ 23.  175.    0. ]]


## Problem 10 — Advanced: stack batches and then unstack safely

You have three mini-batches `B1`, `B2`, `B3`, each shape `(batch, features)`.

1) Stack them into a single array `T` of shape `(3, batch, features)`.
2) Recover the original batches back into a Python list `batches` (length 3) without copying.

Notes:
- Stacking creates a new array, but slicing that stacked array should be a view.
- Use `np.shares_memory` to show `batches[i]` shares memory with `T`.


In [11]:
batch, features = 4, 3
B1 = rng.normal(size=(batch, features))
B2 = rng.normal(size=(batch, features))
B3 = rng.normal(size=(batch, features))

# --- Solution ---
T = np.stack([B1, B2, B3], axis=0)  # (3, batch, features)
batches = [T[i] for i in range(T.shape[0])]  # views into T

print("T.shape:", T.shape)
print("shares_memory(T[0], T):", np.shares_memory(T[0], T))
print("shares_memory(batches[2], T):", np.shares_memory(batches[2], T))

# Checks
assert T.shape == (3, batch, features)
assert len(batches) == 3
assert np.array_equal(batches[0], T[0])
assert np.shares_memory(batches[0], T)
assert np.shares_memory(batches[1], T)
assert np.shares_memory(batches[2], T)


T.shape: (3, 4, 3)
shares_memory(T[0], T): True
shares_memory(batches[2], T): True


## Bonus Problem — Diagnose and fix an axis bug

A student wrote:

```python
X = np.stack([a, b, c], axis=0)
```

They expected `X` to be shape `(n, 3)` (columns), but got `(3, n)`.

Task:
1) Reproduce the issue with `n=5`.
2) Fix it in **two different ways**.
3) Add assertions for both fixes.


In [12]:
n = 5
a = np.arange(n)
b = np.arange(n) + 10
c = np.arange(n) + 100

# --- Solution ---
X_wrong = np.stack([a, b, c], axis=0)
print("X_wrong.shape:", X_wrong.shape)

# Fix 1: stack with axis=1
X1 = np.stack([a, b, c], axis=1)

# Fix 2: column_stack (idiomatic for 1D columns)
X2 = np.column_stack([a, b, c])

print("X1.shape:", X1.shape)
print("X2.shape:", X2.shape)

# Checks
assert X_wrong.shape == (3, n)
assert X1.shape == (n, 3)
assert X2.shape == (n, 3)
assert np.array_equal(X1, X2)
assert np.array_equal(X1[:, 0], a)
assert np.array_equal(X1[:, 1], b)
assert np.array_equal(X1[:, 2], c)


X_wrong.shape: (3, 5)
X1.shape: (5, 3)
X2.shape: (5, 3)
